In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pywt
import csv
import pandas as pd
!pip install mne
import mne
from IPython.display import clear_output
clear_output()

edffilename = "chb10_89"

In [ ]:
raw = mne.io.read_raw_edf('/content/drive/MyDrive/Dataset_Models/dataset/trail/chb12/'+edffilename+'.edf', preload=True)

Extracting EDF parameters from /content/drive/MyDrive/Dataset_Models/dataset/trail/chb12/chb10_89.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-3-060c96b64553>:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf('/content/drive/MyDrive/Dataset_Models/dataset/trail/chb12/'+edffilename+'.edf', preload=True)


Reading 0 ... 1843199  =      0.000 ...  7199.996 secs...


In [ ]:
print(raw.info['ch_names'])

['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']


In [ ]:
raw = raw.drop_channels(["--0", "--1", "--2", "--3", "--4", "T8-P8-1"], on_missing='ignore')
raw = raw.drop_channels([".-0", ".-1", ".-2", ".-3", ".-4", "T8-P8-1"], on_missing='ignore')

In [ ]:
import matplotlib.pyplot as plt
# Applying a Butterworth filter
raw_filtered = raw.copy().filter(l_freq=0.1, h_freq=50, method='iir', iir_params=dict(order=4, ftype='butter'))

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 50 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 50.00 Hz: -6.02, -6.02 dB



In [ ]:
import pandas as pd

data_path = "/content/drive/MyDrive/Dataset_Models/dataset/trail/labelcsv6/"+edffilename+"_partlabel.csv"
data = pd.read_csv(data_path)

label_counts = data['label'].value_counts()
print(label_counts)

label
0.0    6869
0.5     274
1.0      54
Name: count, dtype: int64


In [ ]:
df = pd.DataFrame(columns=['part', 'value', 'label'])

In [ ]:
from scipy.signal import spectrogram
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import numpy as np
from PIL import Image
import _pickle as cPickle
from io import BytesIO

In [ ]:
your_crop_length = 4  # 4 saniye

dflabel = pd.read_csv('/content/drive/MyDrive/Dataset_Models/dataset/trail/labelcsv6/traillabel.csv')
# Check part existence
for index, row in data.iterrows():
    part_exists = row['part'] in dflabel['part'].values
    if(part_exists):
        x = int(row['part'].split('_')[-1])  # 'chb11_92_x' formatındaki x değerini al
        start_time = (x-1)
        end_time = start_time + your_crop_length
        #print(x, "-----" , start_time, "------" , end_time)
        cropped_raw = raw.copy().crop(tmin=start_time, tmax=end_time)
        cropped_raw.copy().resample(sfreq=128)
        # Kanalların sayısını ve zaman uzunluğunu alın
        n_channels, n_times = cropped_raw.get_data().shape

        # Skaleogramları depolamak için bir dizi oluşturun
        spectrograms = []
        images_rgb = []

        # Her bir kanal için skaleogram oluşturma
        for i in range(n_channels):
            data_single = cropped_raw.get_data()[i]  # Tek bir kanalın verisini alın
            #print(data_single)
            sfreq = cropped_raw.info['sfreq']
            frequencies, times, Sxx = spectrogram(data_single, sfreq)
            spectrograms.append(Sxx)
            a = 10 * np.log10(spectrograms[i])
            a = spectrograms[i]
            #print(a)

            plt.figure(figsize=(4, 3))
            ax = plt.gca()  # Geçerli eksen nesnesini al
            ax.imshow(a, aspect="auto", origin="lower")
            ax.set_axis_off()  # Eksenleri kapat

            #image_name =f'kanal_{i+1}.png'
            buffer = BytesIO()
            plt.savefig(buffer, format='png', bbox_inches='tight', transparent=True, pad_inches=0.0)
            buffer.seek(0)
            resim_verisi = buffer.getvalue()
            buffer.close()
            image = Image.open(BytesIO(resim_verisi))

            # Yeni boyutu hesapla
            width, height = image.size
            print(image.size)
            new_size = (width // 2, height // 2)

            # Yeniden boyutlandır
            image = image.resize(new_size)

            image_rgb = np.array(image)
            image_rgb = image_rgb[:, :, :3]
            #print(image_rgb)
            images_rgb.append(image_rgb)
        stacked_image = np.vstack(images_rgb)
        image = Image.fromarray((stacked_image * 255).astype(np.uint8))
        image.save(f"segment_{start_time}-{end_time}.png")
        #print(stacked_image.shape)
        string_holder = '/content/drive/MyDrive/Dataset_Models/dataset/trail/npys6/'+edffilename+'/'+ row['part'] + '.npy'

        #print(stacked_image)
        pil_image = Image.fromarray(stacked_image)

        np.save(string_holder, stacked_image)

        #Veri eklemek için bir satır oluştur
        new_data = {'part': row['part'], 'value': string_holder, 'label': row['label']}

        # DataFrame'e veriyi ekle
        df = pd.concat([df, pd.DataFrame([new_data])], ignore_index=True)
        #break #-------> test için tek eleman alıp break atıyoruz    #TO DO hash
    continue

In [ ]:
# DataFrame'i CSV dosyasına kaydetme
df.to_csv('/content/drive/MyDrive/Dataset_Models/dataset/trail/lastcsv6/'+edffilename+'.csv', index=False)

###BURANIN ÜZERİNİ ÇALIŞTIRICAN